<a href="https://colab.research.google.com/github/JeonJeongumggumg/PyTorch/blob/main/ExamMLPMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np

In [ ]:
train_dataset = datasets.MNIST(root='MNIST_data/',train = True, transform = transforms.ToTensor(),download = True)#학습데이터, 0~255까지 값을 0~1사이 값으로 변환
test_dataset = datasets.MNIST(root='MNIST_data/',train = False, transform = transforms.ToTensor(),download = True)#테스트데이터

In [3]:
print(len(train_dataset))

train_dataset_size = int(len(train_dataset)*0.85)
valid_dataset_size = int(len(train_dataset)*0.15)
train_dataset, valid_dataset = random_split(train_dataset, [train_dataset_size, valid_dataset_size])

print(len(train_dataset), len(valid_dataset), len(test_dataset))


60000
51000 9000 10000


In [8]:
class MNISTModel(nn.Module):
    def __init__(self): #아키텍쳐를 구성하는 다양한 계층을 정의.
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, data):#입력으로 주어진 학습데이터에 대해 피드포워드 수행
        data = self.flatten(data) #입력층
        data = self.fc1(data) #은닉층
        data = self.relu(data) #ReLu(비선형함수)
        data = self.dropout(data)#Dropout 과적합방지
        logits = self.fc2(data)# 출력층
        return logits

In [10]:
BATCH_SIZE = 32
#배치 데이터를 만들기 위해 DataLoader설정
train_dataset_loader = DataLoader(dataset = train_dataset, batch_size = BATCH_SIZE, shuffle = True)
valid_dataset_loader = DataLoader(dataset = valid_dataset, batch_size = BATCH_SIZE, shuffle = True)
test_dataset_loader = DataLoader(dataset = test_dataset, batch_size = BATCH_SIZE, shuffle = True)

In [11]:
model = MNISTModel()

loss_function = nn.CrossEntropyLoss() #CEL에는 손실함수에는 softmax함수를 포함(정답이 0~9> 다중클래스를 분류를 위함)

optimizer = torch.optim.SGD(model.parameters(), lr = 1e-2)

In [18]:
def model_train(dataloader, model, loss_function, optimizer):

  model.train() #신경망을 학습모드로 전환

  train_loss_sum = train_correct = train_total = 0
  total_train_batch = len(dataloader)

  for images, labels in dataloader: #images에는 MNIST이미지 라벨에는 0~9 정답숫자
      x_train = images.view(-1, 28*28)#처음크기에서 784로 전환
      y_train = labels

      outputs = model(x_train) #입력데이터에 대한 예측값을 계산
      loss = loss_function(outputs, y_train) #모델 예측값과 정답과의 오차인 손실함수를 계산

      optimizer.zero_grad() #역전파 코드, 즉 학습이 진행됨에 따라 모델파라미터를 업뎃하면서 최적화
      loss.backward()
      optimizer.step()

      train_loss_sum+= loss.item()

      train_total += y_train.size(0)
      train_correct +=((torch.argmax(outputs, 1)==y_train)).sum().item()

  train_avg_loss = train_loss_sum / total_train_batch #학습데이터 평균오차 계산
  train_avg_accuracy = 100*train_correct / train_total #학습데이터 평균 정확도 계산


  return (train_avg_loss, train_avg_accuracy)


In [32]:
def model_evaluate(dataloader, model, loss_function):

  model.eval() #신경망을 검증모드로 전환

  with torch.no_grad(): #미분하지 않겠음=모델파라미터를 업뎃시키지 않음

      val_loss_sum = 0
      val_correct=0
      val_total = 0

      val_loss_sum = val_correct = vla_total = 0

      total_val_batch = len(dataloader)

      for images, labels in dataloader:

          x_val = images.view(-1, 28*28)
          y_val = labels

          outputs = model(x_val)
          loss = loss_function(outputs, y_val)

          val_loss_sum += loss.item()

          val_total += y_val.size(0)
          val_correct += ((torch.argmax(outputs,1)==y_val)).sum().item()

      val_avg_loss = val_loss_sum / total_val_batch #검증데이터 평균 오차계산
      val_avg_accuracy = 100*val_correct / val_total #검증데이터 정확도 계

      return (val_avg_loss, val_avg_accuracy)


train mode에서 중요한 것

신경망을 학습모드로 변경하고 데이터로더에서 이미지와 정답을 가져와서 벡터로 변경, 오차 구하고, 역전파시켜 모델파라미터를 최적화

train과 evaluate mode 비교

검증모드 = 모델파라미터를 업뎃시키지 않기에 미분이 불필요(torch.no_grad())

train의 역전파코드가 val에는 없음 = 파라미터 업뎃하지않고 현재 신경망의 정확도와 오차만 알려줌

In [38]:
train_loss_list = []
train_accuracy_list = []

val_loss_list = []
val_accuracy_list = []

EPOCHS = 20

for epoch in range(EPOCHS):
  #model train
  train_avg_loss, train_avg_accuracy = model_train(train_dataset_loader, model, loss_function, optimizer)

  train_loss_list.append(train_avg_loss)
  train_accuracy_list.append(train_avg_accuracy)

  #model evaluate

  val_avg_loss, val_avg_accuracy = model_evaluate(valid_dataset_loader, model, loss_function) #optimizer를 쓸 필요가 없음(최적화 불필요), 쓸 경우 인수문제로 typeError가 발견됨

  val_loss_list.append(val_avg_loss)
  val_accuracy_list.append(val_avg_accuracy)

In [39]:
def model_test(dataloader, model):

    model.eval()

    with torch.no_grad(): #test set으로 데이터를 다룰 때에는 gradient를 주면 안된다.

        test_loss_sum = 0
        test_correct=0
        test_total = 0

        total_test_batch = len(dataloader)

        for images, labels in dataloader: # images에는 이미지, labels에는 0-9 숫자

            # reshape input image into [batch_size by 784]
            # label is not one-hot encoded
            x_test = images.view(-1, 28 * 28) #처음 크기는 (batch_size, 1, 28, 28) / 이걸 (batch_size, 784)로 변환
            y_test = labels

            outputs = model(x_test)
            loss = loss_function(outputs, y_test)

            test_loss_sum += loss.item()

            test_total += y_test.size(0)  # label 열 사이즈 같음
            test_correct += ((torch.argmax(outputs, 1)==y_test)).sum().item() # 예측한 값과 일치한 값의 합

        test_avg_loss = test_loss_sum / total_test_batch
        test_avg_accuracy = 100*test_correct / test_total

        print('accuracy:', test_avg_accuracy)
        print('loss:', test_avg_loss)

In [40]:
model_test(test_dataset_loader, model)

accuracy: 98.02
loss: 0.0661373647549502
